In [20]:
import logging
import os
import re
from urllib.parse import urlparse, parse_qs
import tempfile


from bs4 import BeautifulSoup
import jinja2
import requests

from tessa_chef import make_request, get_parsed_html_from_url, make_fully_qualified_url
from tessa_chef import get_license, get_text
from tessa_chef import get_section_filename, create_predictable_zip

from le_utils.constants import content_kinds, file_formats, licenses

import pprint
pp = pprint.PrettyPrinter(indent=4, width=80)

In [21]:
test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108' # EN
# test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82912' # AR

In [22]:

sec_url = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=1.3'

sec_url = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=2'
sec_url = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81239'
get_section_filename(sec_url)



KeyError: 'section'

In [28]:

BG_IMG_RE = re.compile("background-image:url\((.*)\)")
SEND_FACEBOOK_RE = re.compile("sendFacebook\([^,]*,[^,]*,(.*)\)")


def download_module(module_url):
    print('Scapring module...')
    doc = get_parsed_html_from_url(module_url)
    destination = tempfile.mkdtemp()
    print('destination=', destination)
    
    source_id = parse_qs(urlparse(module_url).query)['id'][0]
    raw_title = doc.select_one("head title").text
    title = raw_title.replace('OLCreate:', '')\
            .replace('TESSA_ARABIC', '')\
            .replace('TESSA_Eng', '')\
            .replace('TESSA_Fr', '')\
            .strip()

    contents_dict = dict(
        kind='TessaModuleContentsDict',
        source_id=source_id,
        title=title,
        children=[],
    )

    is_first_section = True
    section_lis = doc.find('ul', class_='oucontent-tree-current-section').find_all('li', recursive=False)
    for section_li in section_lis:            
        section_link = section_li.find('a')
        section_href = section_link['href']
        # TODO: special case for first section --- since it doesn't save section in filename
        #       manually call download_page with filename section_1.html with contents of current page
        if is_first_section:
            section_filename = 'section_1.html'
            is_first_section = False
        else:
            section_filename = get_section_filename(section_href)
        subsections_ul = section_li.find('ul')
        accesshide_span = section_li.find('span', class_='accesshide')
        subsection_lis = subsections_ul.find_all('li')
        if accesshide_span:
            accesshide_span.extract()
        subsections_ul.extract()
        section_title = get_text(section_li)
        
        section_dict = dict(
            title=section_title,
            href=section_href,
            filename=section_filename,
            children=[],
        )
        contents_dict['children'].append(section_dict)
        
        for subsection_li in subsection_lis:
            subsection_link = subsection_li.find('a')
            subsection_href = subsection_link['href']
            subsection_filename = get_section_filename(subsection_href)
            subaccesshide_span = subsection_li.find('span', class_='accesshide')
            if subaccesshide_span:
                subaccesshide_span.extract()
            subsection_title = get_text(subsection_li)
            subsection_dict = dict(
                title=subsection_title,
                href=subsection_href,
                filename=subsection_filename,
            )
            section_dict['children'].append(subsection_dict)

    module_index_tmpl = jinja2.Template(open('../chefdata/templates/module_index.html').read())
    index_contents = module_index_tmpl.render(module=contents_dict)
    print(index_contents[600:800])
    with open(os.path.join(destination, "index.html"), "w") as f:
        f.write(index_contents)

    # download the html content from each section/subsection

    
    return contents_dict

    zip_path = create_predictable_zip(destination)
    print('zip_path =', zip_path)
    #     module_dict = dict(
    #         kind='HTML5AppNode',
    #         source_id=source_id,
    #         title=title,
    #         license=get_license(licenses.CC_BY_NC_SA, copyright_holder='TESSA'),
    #         description='fake descri',
    #         files=[{'file_type':'HTMLZipFile', 'path':zip_path)],
    #     )
    #     return module_dict





def download_page(page_url):
    print('Scapring module page...')
    pass


In [29]:
toc = download_module(test_module)

for section in toc['children']:
    print('  - section:', section['title'], section['filename'])
    for subsection in section['children']:
        print('    - subsection:', subsection['title'], subsection['filename'])


Scapring module...
destination= /var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmpzx35jki7
p://creativecommons.org/ns#" href="./index.html">Module 1: Personal development – how self-esteem impacts on learning</a>
        </span>
        <span class="main-logo">
          <a href="">
       
  - section: Section 1: Ways to explore who pupils are section_1.html
    - subsection: 1. Organising pupils in groups and pairs section_1_1.html
    - subsection: 2. Finding out what pupils think and feel section_1_2.html
    - subsection: 3. Respecting differences section_1_3.html
    - subsection: Resource 1: Similarities and differences section_1_4.html
    - subsection: Resource 2: Asking questions about feelings section_1_5.html
    - subsection: Resource 3: How to conduct a class survey section_1_6.html
    - subsection: Resource 4: Observing your pupils section_1_7.html
    - subsection: Acknowledgements section_1_8.html
  - section: Section 2: Planning physical growth and development sess

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
  <head>
    <meta http-equiv="content-type" content="text/html; charset=UTF-8"/>
    <title>Module 1: Personal development – how self-esteem impacts on learning</title>
    <link rel="stylesheet" href="Shared/styles.css" type="text/css"/>
  </head>
  <body class="mod-oucontent" id="mod-oucontent-view">
    <div id="page">
      <div class="package_header">
        <span class="index-header">
          <a xmlns:cc="http://creativecommons.org/ns#" href="./index.html">Module 1: Personal development – how self-esteem impacts on learning</a>
        </span>
        <span class="main-logo">
          <a href="">
            <img class="rightfloated" src="Shared/main_logo.png" width="444" height="63"/>
          </a>
        </span>
      </div>
      <div id="content">
        <div id="middle-column">
          <div

In [28]:

def passa():

    def download_assets(selector, attr, middleware=None):
        nodes = doc.select(selector)
        for i, node in enumerate(nodes):
            url = make_fully_qualified_url(node[attr])
            filename = "%s_%s" % (i, os.path.basename(url))
            node[attr] = filename
            download_file(url, destination, request_fn=make_request, filename=filename, middleware_callbacks=middleware)

    def js_middleware(content, url, **kwargs):
        # Polyfill window.localStorage as iframes can't access localStorage.
        return content.replace("window.localStorage",
                "({setItem: function(){}, removeItem: function(){}})")

    # Download all static assets.
    # TODO(davidhu): Also download fonts referenced in http://www.africanstorybook.org/css/app.css
    download_assets("img[src]", "src")  # Images
    download_assets("link[href]", "href")  # CSS
    download_assets("script[src]", "src", middleware=js_middleware) # JS

    # Download all background images, e.g. <div style="background-image:url()">
    # (africanstorybook.org uses these for the main picture found on each page
    # of the storybook.)
    bg_img_nodes = doc.select("div[style*=\"background-image:url(\"]")
    for i, node in enumerate(bg_img_nodes):
        style = node["style"]
        match = BG_IMG_RE.search(style)
        if not match:
            continue

        url = make_fully_qualified_url(match.group(1))
        filename = "%s_%s" % (i, os.path.basename(url))
        node["style"] = BG_IMG_RE.sub("background-image:url(%s)" % filename, style)
        download_file(url, destination, request_fn=make_request, filename=filename)

    # Hide the African Storybook header nav bar.
    header = doc.select_one("#headerBar")
    if header:
        header["style"] = "display: none;"
